In [ ]:
#!pip install cv2
#!pip install opencv-python
#!pip install mtcnn
#!pip install tensorflow
#!pip install tensorflow==2.0.0-beta0
!pip install -U numpy

In [ ]:
import cv2
import mtcnn
from mtcnn.mtcnn import MTCNN
import numpy as np
import pandas as pd
import math
import os
import sys

In [ ]:
detector = MTCNN()

In [ ]:
#Function for preprocessing
def find_face(v,dest):
    n_frames = 11
    video = cv2.VideoCapture(v)
    #video = cv2.VideoCapture(v)
    #print(video.isOpened())
    #print(v)
    #print(os.getcwd())
    framerate = video.get(5)
    #print(framerate)
    count=1
    frameindex=1
    try:
        while (video.isOpened()):
            #if(count==2):
             #   break
            frameId = video.get(1)
            #print()
            success, image = video.read()
            if( image is not None ):
                result = detector.detect_faces(image)
                if(len(result)>1):
                    max_area = 0
                    max_idx = -1
                    idx=0
                    for i in range(len(result)):
                        bounding_box = result[i]['box']
                        area=abs((bounding_box[1]-(bounding_box[1] + bounding_box[3]))*(bounding_box[0]-(bounding_box[0]+bounding_box[2])))
                        #print(area)
                        if area>max_area:
                            max_area=area
                            max_idx=i
                       #idx=idx+1
                    #print(max_area)
                    #print(max_idx)
                    bounding_box = result[max_idx]['box']
                    crop_img = image[(bounding_box[1]-28):(bounding_box[1]-28 + bounding_box[3]+56), (bounding_box[0]-28):(bounding_box[0]-28+bounding_box[2]+56)]

                else:
                    bounding_box = result[0]['box']
                    keypoints = result[0]['keypoints']
                ##Creating rectangle
                    cv2.rectangle(image,
                              (bounding_box[0]-25, bounding_box[1]-25),
                              (bounding_box[0]-25+bounding_box[2]+50, bounding_box[1]-25 + bounding_box[3]+50),
                              (0,155,255),2)
                ##Cropping the images

                    crop_img = image[(bounding_box[1]-28):(bounding_box[1]-28 + bounding_box[3]+56), (bounding_box[0]-28):(bounding_box[0]-28+bounding_box[2]+56)]


                # print(crop_img.size())
                crop_img = cv2.resize(crop_img, (152,179), interpolation = cv2.INTER_AREA)
            if (success != True):
                print("Break1")
                break
            if (frameId % math.floor(framerate) == 0):
            #filename = "/content/drive/My Drive/Hack_4_me/Preprocessed_Images/" + "image_" + str(int(frameId / math.floor(framerate))+1) + ".jpg"
                filename = dest +"\\image_" + str(int(frameId / math.floor(framerate))+1) + ".jpg"
                #filename = "C:\\Users\\20180249\\PDEng\\Self\\Hackathon_June\\Preprocessed_Images\\Original"+"\\image_" + str(int(frameId / math.floor(framerate))+1) + ".jpg"

                cv2.imwrite(filename,crop_img)
                frameindex=frameindex+1
            count=count+1
            if frameindex==n_frames:
                print("Break2")
                break
        return True
    except:
        return False

In [ ]:
def process_folders():
    base_dir="C:\\Users\\20190040\\Documents\\Hackathon\\Hack4peace\\"
    folders=["Deepfakes"]
    dest_dir="C:\\Users\\20190040\\Documents\\Hackathon\\Hack4peace\\Preprocessed_Images\\"
    is_face=False
    count = 0
    i=0
    for f in folders:
        os.chdir(base_dir+f)
        videos=os.listdir()
        videos=[l for l in videos if l.split(".")[-1]=='mp4']
        #print(videos)
        for v in videos:
            video_path=base_dir+f+"\\"+v
            destn_path=dest_dir+f+"\\"+v.replace(".mp4","")
            if not os.path.isdir(destn_path):
                os.mkdir(destn_path)
            print('processing video  ',v, 'count: ', count)
            if i >= 201:
                is_face= find_face(v,destn_path)
            print(v,' has face ',is_face)
            if(is_face):
                count+=1
            i+=1

In [ ]:
process_folders()

In [ ]:
count